In [193]:
%load_ext autoreload
%autoreload 2
    
from transformers.generation import LogitsProcessor,LogitsProcessorList
from transformers import AutoModelForCausalLM, AutoTokenizer
import numpy as np
from scipy.stats import gumbel_l
import copy
import torch
import tqdm
import scipy
from scipy.stats import gumbel_l, gumbel_r
import transformers
from evaluate import load
import pickle
from collections import Counter, defaultdict
from sklearn.linear_model import SGDClassifier
# import nn
import torch.nn as nn


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [194]:
model_name = "meta-llama/Meta-Llama-3-8B-Instruct"
model_name = "HuggingFaceTB/SmolLM-1.7B"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name,trust_remote_code=True,torch_dtype=torch.float16,
                                             device_map='auto').eval()

if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

Loading checkpoint shards: 100%|██████████| 2/2 [00:02<00:00,  1.07s/it]


In [195]:
model = model.eval()

In [277]:
def generate_with_logits(model, tokenizer, input_text, max_new_tokens=50, stop_token=None, noise=None):
    
    input_ids = tokenizer.encode(input_text, return_tensors="pt")
    #print([tokenizer.decode(tok) for tok in input_ids])
    # Initialize variables
    generated_ids = input_ids
    past_key_values = None  # for caching the past key/values
    
    for i in range(max_new_tokens):
        with torch.no_grad():
            outputs = model(generated_ids[:, :], past_key_values=past_key_values, use_cache=False)
        
        logits = outputs.logits[:, -1, :]
        #print("argmaxed token:", tokenizer.decode(logits.argmax()))
        if noise is not None:
            logits += noise[i]
        #print("argmaxed token after noise:", tokenizer.decode(logits.argmax()))
        past_key_values = outputs.past_key_values
        next_token_id = torch.argmax(logits, dim=-1).unsqueeze(0)
        generated_ids = torch.cat([generated_ids, next_token_id], dim=-1)
        next_token_id_scalar = next_token_id.item()
        
        generated_token = tokenizer.decode([next_token_id_scalar], skip_special_tokens=True)
        
        if stop_token and generated_token == stop_token:
            break
        if next_token_id_scalar == tokenizer.eos_token_id:
            break
    
    # Decode the full generated sequence
    final_sentence = tokenizer.decode(generated_ids[0], skip_special_tokens=True)
    
    return final_sentence

In [197]:
prompt = tokenizer.bos_token + "Hola a todos!"
generate_with_logits(model, tokenizer, prompt, max_new_tokens=50)

['<|endoftext|>Hola a todos!']
argmaxed token: 

argmaxed token: I
argmaxed token: ’
argmaxed token: m
argmaxed token:  a
argmaxed token:  Spanish
argmaxed token:  teacher
argmaxed token:  and
argmaxed token:  I
argmaxed token: ’
argmaxed token: m
argmaxed token:  going
argmaxed token:  to
argmaxed token:  teach
argmaxed token:  you
argmaxed token:  how
argmaxed token:  to
argmaxed token:  say
argmaxed token:  “
argmaxed token: I
argmaxed token: ’
argmaxed token: m
argmaxed token:  sorry
argmaxed token: ”
argmaxed token:  in
argmaxed token:  Spanish
argmaxed token: .
argmaxed token: 

argmaxed token: I
argmaxed token: ’
argmaxed token: m
argmaxed token:  sorry
argmaxed token:  in
argmaxed token:  Spanish
argmaxed token: 

argmaxed token: The
argmaxed token:  Spanish
argmaxed token:  word
argmaxed token:  for
argmaxed token:  “
argmaxed token: I
argmaxed token: ’
argmaxed token: m
argmaxed token:  sorry
argmaxed token: ”
argmaxed token:  is
argmaxed token:  “
argmaxed token: Lo
argmaxed

'Hola a todos!\nI’m a Spanish teacher and I’m going to teach you how to say “I’m sorry” in Spanish.\nI’m sorry in Spanish\nThe Spanish word for “I’m sorry” is “Lo sient'

In [198]:
from transformers import pipeline
generator = pipeline('text-generation', model=model, tokenizer=tokenizer)
# generate text
generator(prompt, max_length=50, num_return_sequences=1, do_sample=False)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


[{'generated_text': '<|endoftext|>Hola a todos!\nI’m a Spanish teacher and I’m going to teach you how to say “I’m sorry” in Spanish.\nI’m sorry in Spanish\nThe Spanish word for “I’m'}]

In [71]:
inputs = tokenizer.encode(prompt, return_tensors="pt").to("cuda")
outputs = model.generate(inputs)
tokenizer.decode(outputs[0])

'<|endoftext|>Hola a todos!\nI’m a Spanish teacher and I’m going to'

In [280]:
%load_ext autoreload
%autoreload 2
from sampling import counterfactual_generation, counterfactual_generation_batched, sample_from_truncated_gumbel_vectorized, counterfactual_generation_vectorized

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [282]:
vocab_size = len(tokenizer.get_vocab())
noise = counterfactual_generation_vectorized(model, tokenizer, "It was said wa was 2", vocab_size,prompt=None)


['<|endoftext|>', 'It', ' was', ' said', ' wa', ' was', ' ', '2']


In [284]:
generate_with_logits(model, tokenizer, tokenizer.bos_token, max_new_tokens=50, noise=noise)

'It was said wa was 2'

In [234]:
tokenizer.w2i

AttributeError: 'GPT2TokenizerFast' object has no attribute 'w2i'

In [389]:
noised_logits.shape

(1, 5, 49152)

In [394]:
for row in noised_logits[0]:
    print(row.argmax(), tokenizer.decode(row.argmax()))

1589 It
436  was
1137  said
436  was
0 <|endoftext|>


In [216]:
for i in range(10):
    print(counterfactual_generation(model, tokenizer,   "it was claimed", vocab_size,prompt=None))

100%|██████████| 49152/49152 [00:09<00:00, 5330.20it/s]


['it was claimed']


100%|██████████| 49152/49152 [00:09<00:00, 5364.96it/s]


['it was claimed']


 45%|████▌     | 22227/49152 [00:04<00:04, 5424.16it/s]


KeyboardInterrupt: 

In [199]:
def sample_from_truncated_gumbel(cdf_a,b,gumbel,seed=None):
    cdf_b = gumbel.cdf(b)
    #np.random.seed(0) if seed is None else np.random.seed(seed)
    u = np.random.uniform(0, 1)
    if cdf_b < cdf_a:
        cdf_a, cdf_b = cdf_b, cdf_a
    return gumbel.ppf(cdf_a + u * (cdf_b - cdf_a))

sample_from_truncated_gumbel(-10,0,gumbel_l)

nan

In [ ]:
for i in range(10):
    print(counterfactual_generation(model, tokenizer,   "it was claimed that the king has", vocab_size,prompt=None))

In [ ]:
for i in range(10):
    print(counterfactual_generation(model, tokenizer,   "asdsad rhtut dzc a rt", vocab_size,prompt=None))

In [61]:
gumbel_l.ppf(np.random.rand(3))

array([-0.61691634,  0.24225987, -9.07597244])

In [70]:
gumbel_l.cdf(np.ones(5) * (-500.0))

array([7.12457641e-218, 7.12457641e-218, 7.12457641e-218, 7.12457641e-218,
       7.12457641e-218])

In [143]:
def sample_from_truncated_gumbel_vectorized(cdf_a, b):
    cdf_a = np.zeros_like(b)
    #return np.array([sample_from_truncated_gumbel(cdf_a[i], b[i], gumbel_l) for i in range(len(b))])

def sample_from_truncated_gumbel(cdf_a,b,gumbel,u,seed=None):
    cdf_b = gumbel.cdf(b)
    np.random.seed(0) if seed is None else np.random.seed(seed)
    return gumbel.ppf(cdf_a + u * (cdf_b - cdf_a))

u = np.random.rand(3)
cdf_a = np.zeros(3)
b = np.random.rand(3)
cdf_b = gumbel_l.cdf(b)
print(np.array([sample_from_truncated_gumbel(cdf_a[i], b[i], gumbel_l, u[i]) for i in range(len(cdf_b))]))
print(gumbel_l.ppf(cdf_a + u * (cdf_b - cdf_a)))

[-0.99435995 -0.38364933 -9.48239606]
[-0.99435995 -0.38364933 -9.48239606]
